In [0]:
configs = {
    "fs.azure.account.auth.type": "OAuth",
    "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
    "fs.azure.account.oauth2.client.id": "***",
    "fs.azure.account.oauth2.client.secret": "***",
    "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/***/oauth2/token"
}
 
 
# dbutils.fs.mount(
  #  source="abfss://landing-zone-2@ecomadlsrakesh.dfs.core.windows.net",  # container@storageaccount
  #  mount_point="/mnt/ecomdata1",
   # extra_configs=configs
#) 

In [0]:
storage_account = "ecomadlsrakesh"

spark.conf.set(
  f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net",
  "OAuth"
)

spark.conf.set(
  f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net",
  "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider"
)

spark.conf.set(
  f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net",
  "***"
)

spark.conf.set(
  f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net",
  "***"
)

spark.conf.set(
  f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net",
  "https://login.microsoftonline.com/***/oauth2/token"
)


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import col


In [0]:
spark = SparkSession.builder.appName("EcomDataPipeline").getOrCreate()

In [0]:
spark

In [0]:
user_df = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://landing-zone-2@ecomadlsrakesh.dfs.core.windows.net/users-raw-2")


In [0]:
user_df.show(5)

+--------------------+----+---------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+----------------+-----------+
|      identifierHash|type|  country|language|socialNbFollowers|socialNbFollows|socialProductsLiked|productsListed|productsSold|productsPassRate|productsWished|productsBought|gender|civilityGenderId|civilityTitle|hasAnyApp|hasAndroidApp|hasIosApp|hasProfilePicture|daysSinceLastLogin|seniority|seniorityAsMonths|seniorityAsYears|countryCode|
+--------------------+----+---------+--------+-----------------+---------------+-------------------+--------------+------------+----------------+--------------+--------------+------+----------------+-------------+---------+-------------+---------+-----------------+------------------+---------+-----------------+----

In [0]:
buyer_df = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://landing-zone-2@ecomadlsrakesh.dfs.core.windows.net/buyers-raw-2")


In [0]:
buyer_df.show(5)

+-----------+------+---------+-------------+------------+----------+---------------+-------------+-----------------+--------------------+----------------------+------------------+-------------------------+---------------------+-------------------+-------------------+------------------+----------------------+----------------------+---------------------+------------------+------------------+-----------------+---------------------+---------------------+--------------------+---------------+------------------+-------------+-------------+----------------+----------------+
|    country|buyers|topbuyers|topbuyerratio|femalebuyers|malebuyers|topfemalebuyers|topmalebuyers|femalebuyersratio|topfemalebuyersratio|boughtperwishlistratio|boughtperlikeratio|topboughtperwishlistratio|topboughtperlikeratio|totalproductsbought|totalproductswished|totalproductsliked|toptotalproductsbought|toptotalproductswished|toptotalproductsliked|meanproductsbought|meanproductswished|meanproductsliked|topmeanproductsbo

In [0]:
seller_df = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://landing-zone-2@ecomadlsrakesh.dfs.core.windows.net/sellers-raw-2")

In [0]:
seller_df.show(5)

+---------+------+---------+----------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+-----------+-----------+------------------+-------------+-----------+-----------------+-----------------+------------------+
|  country|   sex|nbsellers|meanproductssold|meanproductslisted|meansellerpassrate|totalproductssold|totalproductslisted|meanproductsbought|meanproductswished|meanproductsliked|totalbought|totalwished|totalproductsliked|meanfollowers|meanfollows|percentofappusers|percentofiosusers|     meanseniority|
+---------+------+---------+----------------+------------------+------------------+-----------------+-------------------+------------------+------------------+-----------------+-----------+-----------+------------------+-------------+-----------+-----------------+-----------------+------------------+
|Allemagne|Female|      116|            4.03|              2.72|             27.33|           

In [0]:
countries_df = spark.read.format("parquet") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("abfss://landing-zone-2@ecomadlsrakesh.dfs.core.windows.net/countries-raw-2")

In [0]:
countries_df.show(5)

+---------+-------+----------+--------------+------------------+---------------------+-------------+-----------+----------------+--------------+----------------+-------------+--------------------+-----------------+----------------------+-------------------+-------------------+---------------------+--------------------+--------------------+---------------+------------------+-------------+-------------+----------------+----------------+
|  country|sellers|topsellers|topsellerratio|femalesellersratio|topfemalesellersratio|femalesellers|malesellers|topfemalesellers|topmalesellers|countrysoldratio|bestsoldratio|toptotalproductssold|totalproductssold|toptotalproductslisted|totalproductslisted|topmeanproductssold|topmeanproductslisted|    meanproductssold|  meanproductslisted|meanofflinedays|topmeanofflinedays|meanfollowers|meanfollowing|topmeanfollowers|topmeanfollowing|
+---------+-------+----------+--------------+------------------+---------------------+-------------+-----------+----------